In [2]:
import openpyxl
from collections import namedtuple
from operator import attrgetter
from itertools import groupby, izip

In [26]:
from process_spreadsheet import dim_processor, derive_domain_profile, derive_time_usage_profile

In [12]:
class RecordReader(object):
    def __init__(self, title_row, entries):
        self._entries = entries
        self._title_row = tuple(cell.value for cell in title_row)
        self._find_indexes()
        self._record_factory = namedtuple('Request', 
                                          self._to_attrs())
    def __call__(self, row):
        args = tuple(row[index].value for index in self._indices)
        return self._record_factory(*args)
        
    def _find_indexes(self):
        self._indices = []
        for entry in self._entries:
            self._indices.append(self._title_row.index(entry))

    def _to_attrs(self):
        result = []
        for entry in self._entries:
            lc = entry.lower()
            result.append(lc.replace(' ', '_'))
        return result
        

In [5]:
def still_valid(row):
    return row[26].value == None or 'DELETE' not in row[26].value

In [ ]:
source='/home/vagrant/Downloads/CMIP6_datareq_UKESM_mappings.xlsx'
wb=openpyxl.load_workbook(source)

ws=wb.get_sheet_by_name('Diagnostics')

In [22]:
rows = ws.rows

titles = next(rows) # initial row
titles[0].value=u'cmor_label' # replace the 'None'
record = RecordReader(titles, ['cmor_label', 'miptable', 
                               'cell_methods', 'dimension',
                              'frequency', 'STASH codes needed',
                              'Variable_mapping', 'Plan'])
print [cell.value for cell in titles]
records = [record(row) for row in rows if still_valid(row)]

[u'cmor_label', u'title', u'miptable', u'cf_std_name', u'description', u'cell_methods', u'dimension', u'units', u'positive', u'realm', u'priority', u'requesting_mips', u'frequency', u'last_update', u'UKESM_component', u'Variable_mapping', u'PP_constraint', u'Model_units', u'Model_positive', u'Stream', u'Plan', u'Ticket', u'Comment (this goes into file metadata)', u"Notes (this doesn't go in the metadata)", u'Implemented', u'Model run test data', u'Dreq upgrade 01.bet.26 to 01.beta.37', u'duplicate key', u'Duplicate?', u'Fingerprint', u'STASH codes needed', u'Known issues']


In [56]:
def included(record):
    return ('fx' not in record.miptable and 
            record.stash_codes_needed and
            record.plan and
            record.plan != 'do not produce')

wanted = filter(included, records)

In [57]:
dimensions =list(dim_processor(record.dimension) for record in wanted)
domain_results, _ = derive_domain_profile(dimensions)
unknowns = (record for record, result in izip(wanted, domain_results) if result.lower() == 'unknown')

get_dims = attrgetter('dimension')
for dims, recs in groupby(sorted(unknowns, key=get_dims), get_dims):
    print dims
    for rec in recs:
        print ' ', rec.miptable, rec.cmor_label, rec.variable_mapping

alevel site time1
  CFsubhr cl m01s02i261
  CFsubhr cli m01s02i309
  CFsubhr clw m01s02i308
  CFsubhr edt m01s03i472
  CFsubhr evu m01s03i471
  CFsubhr hur m01s30i113
  CFsubhr hus m01s00i010
  CFsubhr pfull m01s00i408
  CFsubhr ta m01s30i111
  CFsubhr tnhus m01s30i182 / TIMESTEP
  CFsubhr tnhusa m01s12i182 / TIMESTEP
  CFsubhr tnhusc m01s05i162 / TIMESTEP
  CFsubhr tnhusmp ( m01s01i182+m01s02i182+m01s03i182+m01s04i142+m01s04i182+m01s05i182+m01s16i162+m01s16i182+m01s35i025 ) / TIMESTEP
  CFsubhr tnhusscpbl (m01s03i182+m01s04i142+m01s04i182+m01s05i182-m01s05i162+m01s16i162+m01s16i182) / TIMESTEP
  CFsubhr tnt m01s30i181 / TIMESTEP
  CFsubhr tnta m01s12i181 / TIMESTEP
  CFsubhr tntc m01s05i161 / TIMESTEP
  CFsubhr tntmp (m01s01i181 + m01s02i181 + m01s03i181 + m01s04i141 + m01s04i181 + m01s05i181 + m01s06i181 + m01s16i161+ m01s16i181+m01s35i024) / TIMESTEP
  CFsubhr tntr (m01s01i161 + m01s02i161) / TIMESTEP
  CFsubhr tntscpbl (m01s03i181+m01s04i141+m01s04i181+m01s16i161+m01s16i181+m01s01i

In [58]:
def time_usage(rec):
    res = derive_time_usage_profile(rec.miptable,
                                    [rec.frequency],
                                    [rec.cell_methods],
                                    dbg=False)
    return res[0]

time_result = [time_usage(record) for record in wanted]

print 'unknown time profiles'
for result, rec in izip(time_result, wanted):
    if result[0].lower() == 'unknown':
        print ' ', rec.miptable, rec.cmor_label, rec.variable_mapping, rec.frequency, rec.cell_methods


unknown time profiles
  Emon pfull m01s00i408 mon None
  Emon phalf m01s00i407 mon None
  6hrLev pmlev m01s00i408 6hr None
  6hrPlev ta m01s30i294/m01s30i304 6hr None
  Emon tntlw m01s02i161 / TIMESTEP mon None
  6hrPlev ua m01s30i201/m01s30i301 6hr None
  6hrPlev va m01s30i202/m01s30i301 6hr None


In [59]:
print 'uknown usage profiles'
for result, rec in izip(time_result, wanted):
    if result[1].lower() == 'unknown':
        print ' ', rec.miptable, rec.plan, rec.cmor_label, rec.variable_mapping, rec.frequency, rec.cell_methods

uknown usage profiles
  IyrAnt post-process acabf SUM_icetiles(m01s08i578*m01s03i317) yr area: time: mean where ice_shelf
  IyrGre post-process acabf SUM_icetiles(m01s08i578*m01s03i317) yr area: time: mean where ice_shelf
  IyrAnt post-process areacelli SUM_icetiles(m01s03i317*m01s03i395) yr time: mean
  IyrGre post-process areacelli SUM_icetiles(m01s03i317*m01s03i395) yr time: mean
  Eyr available cSoil m01s19i016 yr time: point
  Eyr available cVeg m01s19i002 yr time: point
  Eyr post-process fracLut lutFrac(m01s19i013,m01s00i505) yr area: mean where landUse time: point
  IyrAnt post-process orog m01s00i033 yr area: time: mean where ice_shelf
  IyrGre post-process orog m01s00i033 yr area: time: mean where ice_shelf
  E1hrClimMon available rlut m01s03i332 1hrClimMon area: mean time: mean within days time: mean over days
  E1hrClimMon available rlutcs m01s02i206+m01s03i332-m01s02i205 1hrClimMon area: mean time: mean within days time: mean over days
  E1hrClimMon available rsdt m01s01i2